### The code is based on https://github.com/dgd45125/LPPxORCxEN-CN/tree/main/GLM_analyses

# Whole Brain Analysis: GLM second level analysis

# Imports

In [ ]:
import pandas as pd

import numpy as np

from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.image import threshold_img
from nilearn.glm import threshold_stats_img
from nilearn import surface
from nilearn import datasets
from nilearn.reporting import get_clusters_table
from nilearn.image import resample_to_img
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
#from nilearn.maskers import NiftiLabelsMasker

import glob

from natsort import natsorted

import matplotlib.pyplot as plt

In [ ]:
sample = './nii/sub30/swu20211222114602_BCCWJ30_7801_EPI_BCCWJ_A_s501.nii'

# Atlas (masker)

In [ ]:
Yeo = './Yeo2011_7Networks_MNI152_FreeSurferConformed1mm_LiberalMask.nii.gz'
resampled_Yeo = resample_to_img(source_img=Yeo, target_img=sample,interpolation='nearest')

#plotting.plot_roi(resampled_Yeo,cut_coords=(-0,18,22))

parcelation_masker = NiftiLabelsMasker(labels_img=resampled_Yeo)
time_series = parcelation_masker.fit_transform(sample)
resampled_yeo_binary = parcelation_masker.inverse_transform([[1,1,1,1,1,1,1]])
plotting.plot_roi(resampled_yeo_binary,cut_coords=(0,18,22))

# Word Rate

In [ ]:
bccwj_word_rate_files = natsorted(glob.glob(f"./glm/word_rate/*word_rate*"))
design_matrix = pd.DataFrame([1] * len(bccwj_word_rate_files), columns=['intercept'])

bccwj_word_rate_model = SecondLevelModel(smoothing_fwhm=8.0, mask_img=resampled_yeo_binary)
bccwj_word_rate_model = bccwj_word_rate_model.fit(bccwj_word_rate_files, design_matrix=design_matrix)

bccwj_word_rate_map = bccwj_word_rate_model.compute_contrast(output_type= 'z_score')

In [ ]:
bccwj_word_rate_thresholded, bccwj_word_rate_threshold = threshold_stats_img(bccwj_word_rate_map, 
                                                                             alpha=.05,
                                                                             height_control = 'fdr',
                                                                             two_sided=False,
                                                                             cluster_threshold=50)
print(bccwj_word_rate_threshold)
plotting.plot_glass_brain(bccwj_word_rate_thresholded,plot_abs=False, 
                          threshold = bccwj_word_rate_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('wordrate')

In [ ]:
table = get_clusters_table(stat_img = bccwj_word_rate_thresholded,
                           stat_threshold=bccwj_word_rate_threshold,
                           two_sided=True)
table

In [ ]:
table = get_clusters_table(stat_img = bccwj_word_rate_thresholded,
                           stat_threshold=bccwj_word_rate_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
pip install atlasreader

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_word_rate_thresholded, cluster_extent=3)

In [ ]:
import pandas as pd
roi_word_rate = pd.read_csv('./atlasreader_clusters.csv')
roi_word_rate

In [ ]:
print(roi_word_rate.to_latex())

In [ ]:
roi_word_rate_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_word_rate_peak

In [ ]:
print(roi_word_rate_peak.to_latex())

# Word Length

In [ ]:
bccwj_word_length_files = natsorted(glob.glob(f"./glm/word_length/*word_length*"))
design_matrix = pd.DataFrame([1] * len(bccwj_word_length_files), columns=['intercept'])

bccwj_word_length_model = SecondLevelModel(smoothing_fwhm=8.0,
                           mask_img=resampled_yeo_binary) 
bccwj_word_length_model = bccwj_word_length_model.fit(bccwj_word_length_files, design_matrix=design_matrix)
bccwj_word_length_map = bccwj_word_length_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_word_length_thresholded, bccwj_word_length_threshold = threshold_stats_img(bccwj_word_length_map, 
                                                                             alpha=.05,
                                                                             height_control = 'fdr',
                                                                             two_sided=False,
                                                                             cluster_threshold=100)
print(bccwj_word_length_threshold)
plotting.plot_glass_brain(bccwj_word_length_thresholded,plot_abs=False, 
                          threshold = bccwj_word_length_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('wordlength')

In [ ]:
table = get_clusters_table(stat_img = bccwj_word_length_thresholded,
                           stat_threshold=bccwj_word_length_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
pip install atlasreader

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_word_length_thresholded, cluster_extent=3)

In [ ]:
import pandas as pd
roi_word_length = pd.read_csv('./atlasreader_clusters.csv')
roi_word_length

In [ ]:
print(roi_word_length.to_latex())

In [ ]:
roi_word_length_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_word_length_peak

In [ ]:
print(roi_word_length_peak.to_latex())

# Word Frequency

In [ ]:
bccwj_word_freq_files = natsorted(glob.glob(f"./glm/word_freq/*word_freq*"))
design_matrix = pd.DataFrame([1] * len(bccwj_word_freq_files), columns=['intercept'])

bccwj_word_freq_model = SecondLevelModel(smoothing_fwhm=8.0,
                          mask_img=resampled_yeo_binary) 
bccwj_word_freq_model = bccwj_word_freq_model.fit(bccwj_word_freq_files, design_matrix=design_matrix)

bccwj_word_freq_map = bccwj_word_freq_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_word_freq_thresholded, bccwj_word_freq_threshold = threshold_stats_img(bccwj_word_freq_map, 
                                                                             alpha=.05,
                                                                             height_control = 'fdr',
                                                                             two_sided=True,
                                                                             cluster_threshold=100)
print(bccwj_word_freq_threshold)
plotting.plot_glass_brain(bccwj_word_freq_thresholded,plot_abs=False, 
                          threshold = bccwj_word_freq_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('wordfreq')

In [ ]:
table = get_clusters_table(stat_img = bccwj_word_freq_thresholded,
                           stat_threshold=bccwj_word_freq_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_word_freq_thresholded, cluster_extent=3)

In [ ]:
roi_word_freq_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_word_freq_peak

In [ ]:
print(roi_word_freq_peak.to_latex())

# Sentid

In [ ]:
bccwj_sentid_files = natsorted(glob.glob(f"./glm/sent_id/*sent_id*"))
design_matrix = pd.DataFrame([1] * len(bccwj_sentid_files), columns=['intercept'])

bccwj_sentid_model = SecondLevelModel(smoothing_fwhm=8.0,  mask_img=resampled_yeo_binary
                            ) 
bccwj_sentid_model = bccwj_sentid_model.fit(bccwj_sentid_files, design_matrix=design_matrix)

bccwj_sentid_map = bccwj_sentid_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_sentid_thresholded, bccwj_sentid_threshold = threshold_stats_img(bccwj_sentid_map, 
                                                                             alpha=.05,
                                                                             height_control = 'fdr',
                                                                             two_sided=False,
                                                                             cluster_threshold=100)
print(bccwj_sentid_threshold)
plotting.plot_glass_brain(bccwj_sentid_thresholded,plot_abs=False, 
                          threshold = bccwj_sentid_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('sentid')

In [ ]:
table = get_clusters_table(stat_img = bccwj_sentid_thresholded,
                           stat_threshold=bccwj_sentid_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_sentid_thresholded, cluster_extent=3)

In [ ]:
roi_sentid_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_sentid_peak

In [ ]:
print(roi_sentid_peak.to_latex())

# Sentpos

In [ ]:
bccwj_sentpos_files = natsorted(glob.glob(f"./glm/sent_pos/*sentpos*"))
design_matrix = pd.DataFrame([1] * len(bccwj_sentpos_files), columns=['intercept'])

bccwj_sentpos_model = SecondLevelModel(smoothing_fwhm=8.0, mask_img=resampled_yeo_binary
                            )
bccwj_sentpos_model = bccwj_sentpos_model.fit(bccwj_sentpos_files, design_matrix=design_matrix)

bccwj_sentpos_map = bccwj_sentpos_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_sentpos_thresholded, bccwj_sentpos_threshold = threshold_stats_img(bccwj_sentpos_map, 
                                                                             alpha=.05,
                                                                             #height_control = 'fdr',
                                                                             two_sided=False,
                                                                             cluster_threshold=100)
print(bccwj_sentpos_threshold)
plotting.plot_glass_brain(bccwj_sentpos_thresholded,plot_abs=False, 
                          threshold = bccwj_sentpos_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('sentpos')

In [ ]:
table = get_clusters_table(stat_img = bccwj_sentpos_thresholded,
                           stat_threshold=bccwj_sentpos_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_sentpos_thresholded, cluster_extent=3)

In [ ]:
roi_sentpos_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_sentpos_peak

In [ ]:
print(roi_sentpos_peak.to_latex())

# Ngram-five

In [ ]:
bccwj_ngram_files = natsorted(glob.glob(f"./glm/ngram/*ngram*"))
design_matrix = pd.DataFrame([1] * len(bccwj_ngram_files), columns=['intercept'])

bccwj_ngram_model = SecondLevelModel(smoothing_fwhm=8.0,  mask_img=resampled_yeo_binary
                            )
bccwj_ngram_model = bccwj_ngram_model.fit(bccwj_ngram_files, design_matrix=design_matrix)

bccwj_ngram_map = bccwj_ngram_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_ngram_thresholded, bccwj_ngram_threshold = threshold_stats_img(bccwj_ngram_map, 
                                                                     alpha=.05, 
                                                                     two_sided=False, 
                                                                     cluster_threshold=100,
                                                                     height_control= 'fdr'
                                                                  )
print(bccwj_ngram_threshold)
plotting.plot_glass_brain(bccwj_ngram_thresholded,plot_abs=False, 
                          threshold = bccwj_ngram_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('ngram')

In [ ]:
table = get_clusters_table(stat_img = bccwj_ngram_thresholded,
                           stat_threshold=bccwj_ngram_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_ngram_thresholded, cluster_extent=3)

In [ ]:
roi_ngram_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_ngram_peak

In [ ]:
print(roi_ngram_peak.to_latex())

# LSTM

In [ ]:
bccwj_lstm_files = natsorted(glob.glob(f"./glm/lstm/*lstm*"))
design_matrix = pd.DataFrame([1] * len(bccwj_lstm_files), columns=['intercept'])

bccwj_lstm_model = SecondLevelModel(smoothing_fwhm=8.0,  mask_img=resampled_yeo_binary
                            )
bccwj_lstm_model = bccwj_lstm_model.fit(bccwj_lstm_files, design_matrix=design_matrix)

bccwj_lstm_map = bccwj_lstm_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_lstm_thresholded, bccwj_lstm_threshold = threshold_stats_img(bccwj_lstm_map, 
                                                                     alpha=.05, 
                                                                     two_sided=False, 
                                                                     cluster_threshold=50,
                                                                     #height_control= 'fdr'
                                                                  )
print(bccwj_lstm_threshold)
plotting.plot_glass_brain(bccwj_lstm_thresholded,plot_abs=False, 
                          threshold = bccwj_lstm_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('lstm')

In [ ]:
table = get_clusters_table(stat_img = bccwj_lstm_thresholded,
                           stat_threshold=bccwj_lstm_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_lstm_thresholded, cluster_extent=3)

In [ ]:
roi_lstm_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_lstm_peak

In [ ]:
print(roi_lstm_peak.to_latex())

# surp_RNNG_TD

In [ ]:
bccwj_rnng_td_files = natsorted(glob.glob(f"./glm/surp_rnng_td/*rnng_td*"))
design_matrix = pd.DataFrame([1] * len(bccwj_rnng_td_files), columns=['intercept'])

bccwj_rnng_td_model = SecondLevelModel(smoothing_fwhm=8.0,  mask_img=resampled_yeo_binary
                            )
bccwj_rnng_td_model = bccwj_rnng_td_model.fit(bccwj_rnng_td_files, design_matrix=design_matrix)

bccwj_rnng_td_map = bccwj_rnng_td_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_rnng_td_thresholded, bccwj_rnng_td_threshold = threshold_stats_img(bccwj_rnng_td_map, 
                                                                     alpha=.05, 
                                                                     two_sided=False, 
                                                                     cluster_threshold=100,
                                                                     #height_control='fdr'
                                                                  )
print(bccwj_rnng_td_threshold)
plotting.plot_glass_brain(bccwj_rnng_td_thresholded,plot_abs=False, 
                          threshold = bccwj_rnng_td_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('surprnngtd')

In [ ]:
table = get_clusters_table(stat_img = bccwj_rnng_td_thresholded,
                           stat_threshold=bccwj_rnng_td_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_rnng_td_thresholded, cluster_extent=3)

In [ ]:
roi_rnng_td_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_rnng_td_peak

In [ ]:
print(roi_rnng_td_peak.to_latex())

# surp_RNNG_LC

In [ ]:
bccwj_rnng_lc_files = natsorted(glob.glob(f"./glm/surp_rnng_lc/*rnng_lc*"))
design_matrix = pd.DataFrame([1] * len(bccwj_rnng_lc_files), columns=['intercept'])

bccwj_rnng_lc_model = SecondLevelModel(smoothing_fwhm=8.0,  mask_img=resampled_yeo_binary
                            )
bccwj_rnng_lc_model = bccwj_rnng_lc_model.fit(bccwj_rnng_lc_files, design_matrix=design_matrix)

bccwj_rnng_lc_map = bccwj_rnng_lc_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_rnng_lc_thresholded, bccwj_rnng_lc_threshold = threshold_stats_img(bccwj_rnng_lc_map, 
                                                                     alpha=.05, 
                                                                     two_sided=False, 
                                                                     cluster_threshold=100,
                                                                     #height_control='fdr'
                                                                  )
print(bccwj_rnng_lc_threshold)
plotting.plot_glass_brain(bccwj_rnng_lc_thresholded,plot_abs=False, 
                          threshold = bccwj_rnng_lc_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('surprnnglc')

In [ ]:
table = get_clusters_table(stat_img = bccwj_rnng_lc_thresholded,
                           stat_threshold=bccwj_rnng_lc_threshold,
                           two_sided=True)

table = table.round({'X':0,'Y':0,'Z':0,'Peak Stat':2})
table.drop(columns='Cluster ID',inplace=True)
table=table[['Cluster Size (mm3)','X','Y','Z','Peak Stat']]
table

In [ ]:
print(table.to_latex())

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_rnng_lc_thresholded, cluster_extent=3)

In [ ]:
roi_rnng_lc_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_rnng_lc_peak

In [ ]:
print(roi_rnng_lc_peak.to_latex())

## dis_RNNG_TD

In [ ]:
bccwj_dis_rnng_td_files = natsorted(glob.glob(f"./glm/dis_rnng_td/*dis_rnng_td*"))
design_matrix = pd.DataFrame([1] * len(bccwj_dis_rnng_td_files), columns=['intercept'])

bccwj_dis_rnng_td_model = SecondLevelModel(smoothing_fwhm=8.0,mask_img=resampled_yeo_binary
                            ) 
bccwj_dis_rnng_td_model = bccwj_dis_rnng_td_model.fit(bccwj_dis_rnng_td_files, design_matrix=design_matrix)

bccwj_dis_rnng_td_map = bccwj_dis_rnng_td_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_dis_rnng_td_thresholded, bccwj_dis_rnng_td_threshold = threshold_stats_img(bccwj_dis_rnng_td_map, 
                                                                             alpha=.05,
                                                                             height_control = 'fdr',
                                                                             two_sided=False,
                                                                             cluster_threshold=100)
print(bccwj_dis_rnng_td_threshold)
plotting.plot_glass_brain(bccwj_dis_rnng_td_thresholded,plot_abs=False, 
                          threshold = bccwj_dis_rnng_td_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('disrnngtd')

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_dis_rnng_td_thresholded, cluster_extent=3)

In [ ]:
roi_dis_rnng_td_peak = pd.read_csv('./atlasreader_peaks.csv')
roi_dis_rnng_td_peak

In [ ]:
print(roi_dis_rnng_td_peak.to_latex())

##  dis_RNNG_LC 

In [ ]:
bccwj_dis_rnng_lc_files = natsorted(glob.glob(f"./glm/dis_rnng_lc/*dis_rnng_lc*"))
design_matrix = pd.DataFrame([1] * len(bccwj_dis_rnng_lc_files), columns=['intercept'])

bccwj_dis_rnng_lc_model = SecondLevelModel(smoothing_fwhm=8.0,mask_img=resampled_yeo_binary
                            ) 
bccwj_dis_rnng_lc_model = bccwj_dis_rnng_lc_model.fit(bccwj_dis_rnng_lc_files, design_matrix=design_matrix)

bccwj_dis_rnng_lc_map = bccwj_dis_rnng_lc_model.compute_contrast(output_type='z_score')

In [ ]:
bccwj_dis_rnng_lc_thresholded, bccwj_dis_rnng_lc_threshold = threshold_stats_img(bccwj_dis_rnng_lc_map, 
                                                                             alpha=.05,
                                                                             #height_control = 'fdr',
                                                                             two_sided=False,
                                                                             cluster_threshold=100)
print(bccwj_dis_rnng_lc_threshold)
plotting.plot_glass_brain(bccwj_dis_rnng_lc_thresholded,plot_abs=False, 
                          threshold = bccwj_dis_rnng_lc_threshold,
                          display_mode='lyrz',
                          colorbar=True)
plt.savefig('disrnnglc')

In [ ]:
# https://github.com/miykael/atlasreader/blob/master/notebooks/atlasreader.ipynb
from atlasreader import create_output
create_output(bccwj_dis_rnng_lc_thresholded, cluster_extent=3)

In [ ]:
roi_dis_rnng_lc_peak = pd.read_csv('/Users/osekilab1/Desktop/BCCWJ-fMRI-ts/notebooks/atlasreader_peaks.csv')
roi_dis_rnng_lc_peak